In [ ]:
!pip install openmim
!pip install -q /kaggle/input/frozen-packages-mmdetection/mmcv-2.0.1-cp310-cp310-linux_x86_64.whl

!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

%cd ..
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose
!pip install -e .

!mkdir checkpoint
!mkdir outputs
!mkdir data


In [ ]:
from IPython import display
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('MMCV版本', mmcv.__version__)
%cd /kaggle/working/mmdetection
import mmdet
print('mmdetection版本', mmdet.__version__)
%cd /kaggle/working/mmpose
import mmpose
print('mmpose版本', mmpose.__version__)
print('CUDA版本', get_compiling_cuda_version())
print('编译器版本', get_compiler_version())

In [ ]:
import numpy as np
import random
import joblib
from PIL import Image
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                        process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
from mmseg.apis import init_segmentor, inference_segmentor
import os
import math
import torch, torchvision

In [ ]:
IMAGE_PATH = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
JSON_PATH =  '/kaggle/input/hrnettest3333/rebatrain_keypoints.json'


VAL_IMAGE_PATH = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
VAL_JSON_PATH =  '/kaggle/input/hrnettest3333/rebatrain_keypoints.json'

In [ ]:
!ls /kaggle/working/mmpose/data

In [ ]:
!rm -rf /kaggle/working/mmpose/data/*

In [ ]:
import os
import io
import json
import shutil
import random
import numpy as np
from pathlib import Path

from PIL import Image
from tqdm import tqdm
from mmengine import Config

from pycocotools.coco import COCO

import os
with open(JSON_PATH) as jfff:
    ccc = json.load(jfff)
i_names = [dx['image_file_name'] for dx in ccc]
def separate_files(og_folder, trans_folder):
    
    image_folder = os.path.join(trans_folder, 'images')
    ann_folder = os.path.join(trans_folder, 'ann')
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(ann_folder, exist_ok=True)
    for file in tqdm(os.listdir(data_folder)):
        if file not in i_names:
            continue
        if file.endswith('.jpg'):
            source_path = os.path.join(og_folder, file)
            target_path = os.path.join(image_folder, file)
            shutil.copy(source_path, target_path)
       
    
    
data_folder = IMAGE_PATH
trans_folder = './data'

separate_files(data_folder, trans_folder)

import json
def read_file_as_list(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        key_point = [int(num) for num in content.split()]
        key_num = key_point[0]
        key_point.pop(0)
        for i in range(2, len(key_point) + len(key_point)//2, 2 + 1):
            key_point.insert(i, 2)
    return key_num,key_point

def get_image_size(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height


def read_file_as_list(file_path):
    key_point = file_path['keypoints'] 
    key_num = 18
    return key_num,key_point


def coco_structure(train_file,train_image_dir,val_file,val_image_dir):
    coco = dict()
    coco['images'] = []
    coco['annotations'] = []
    coco['categories'] = []
    coco['categories'].append(dict(supercategory = 'person',id = 1,name = 'person',
                               keypoints = ['forehead', 'nose',  'neck', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'center_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle','left_hand','right_hand'],
                               skeleton = [(0, 1),  # forehead to nose
                                            (1, 2),  # nose to neck
                                            (2, 3),  # neck to left shoulder
                                            (2, 4),  # neck to right shoulder
                                            (3, 5),  # left shoulder to left elbow
                                            (4, 6),  # right shoulder to right elbow
                                            (5, 7),  # left elbow to left wrist
                                            (6, 8),  # right elbow to right wrist
                                            (2, 9),  # neck to center hip
                                            (9, 10),  # center hip to left hip
                                            (9, 11),  # center hip to right hip
                                            (10, 12),  # left hip to left knee
                                            (11, 13),  # right hip to right knee
                                            (12, 14),  # left knee to left ankle
                                            (13, 15),  # right knee to right ankle
                                            (7, 16),  # left wrist to left hand
                                            (8, 17)  # right wrist to right hand
                                          ]))
    with open(train_file,'r') as file:
        ann_list = json.load(file)
            
    id = 0
    for ann_file in tqdm(ann_list):
        key_num,key_point = read_file_as_list(ann_file)
        img_file_name = ann_file['image_file_name']
        img_id = ann_file['image_id']
        if key_num == 18:
            image_name = img_file_name
            image_id = img_id
            height, width = get_image_size(os.path.join(train_image_dir, image_name))
            image = {"id": id, "file_name": image_name, "height": height, "width": width}
            coco['images'].append(image)
            key_dict = dict(category_id = 1, segmentation = [], iscrowd = 0, image_id = id, 
                    id = id, bbox = [0, 0, width, height], area = width * height, num_keypoints = key_num, keypoints = key_point)
            coco['annotations'].append(key_dict)
            id = id + 1
    with open(val_file,'r') as file:
        ann_list = json.load(file)
            
    id = 0
    for ann_file in tqdm(ann_list):
        key_num,key_point = read_file_as_list(ann_file)
        img_file_name = ann_file['image_file_name']
        img_id = ann_file['image_id']
        if key_num == 18:
            image_name = img_file_name
            image_id = img_id
            height, width = get_image_size(os.path.join(val_image_dir, image_name))
            image = {"id": id, "file_name": image_name, "height": height, "width": width}
            coco['images'].append(image)
            key_dict = dict(category_id = 1, segmentation = [], iscrowd = 0, image_id = id, 
                    id = id, bbox = [0, 0, width, height], area = width * height, num_keypoints = key_num, keypoints = key_point)
            coco['annotations'].append(key_dict)
            id = id + 1
    print("Total annoatations: ",len(coco['annotations']))
    return coco

train_file = coco_structure(JSON_PATH,IMAGE_PATH,VAL_JSON_PATH,VAL_IMAGE_PATH)

output_file_path =  './data/annotations.json'
with open(output_file_path, "w", encoding="utf-8") as output_file:
    json.dump(train_file, output_file, ensure_ascii=True, indent=4)

    

def split_coco_dataset(coco_json_path: str, save_dir: str, ratios: list,
                       shuffle: bool, seed: int):
    if not Path(coco_json_path).exists():
        raise FileNotFoundError(f'Can not not found {coco_json_path}')

    if not Path(save_dir).exists():
        Path(save_dir).mkdir(parents=True)

    ratios = np.array(ratios) / np.array(ratios).sum()

    if len(ratios) == 2:
        ratio_train, ratio_test = ratios
        print("Ratio train:",ratio_train)
        ratio_val = 0
        train_type = 'trainval'
    elif len(ratios) == 3:
        ratio_train, ratio_val, ratio_test = ratios
        train_type = 'train'
    else:
        raise ValueError('ratios must set 2 or 3 group!')

    coco = COCO(coco_json_path)
    coco_image_ids = coco.getImgIds()

    val_image_num = int(len(coco_image_ids) * ratio_val)
    test_image_num = int(len(coco_image_ids) * ratio_test)
    train_image_num = len(coco_image_ids) - val_image_num - test_image_num
    print('Split info: ====== \n'
          f'Train ratio = {ratio_train}, number = {train_image_num}\n'
          f'Val ratio = {ratio_val}, number = {val_image_num}\n'
          f'Test ratio = {ratio_test}, number = {test_image_num}')

    seed = int(seed)
    if seed != -1:
        print(f'Set the global seed: {seed}')
        np.random.seed(seed)

    if shuffle:
        print('shuffle dataset.')
        random.shuffle(coco_image_ids)

    train_image_ids = coco_image_ids[:train_image_num]
    if val_image_num != 0:
        val_image_ids = coco_image_ids[train_image_num:train_image_num +
                                       val_image_num]
    else:
        val_image_ids = None
    test_image_ids = coco_image_ids[train_image_num + val_image_num:]

    categories = coco.loadCats(coco.getCatIds())
    for img_id_list in [train_image_ids, val_image_ids, test_image_ids]:
        if img_id_list is None:
            continue

        img_dict = {
            'images': coco.loadImgs(ids=img_id_list),
            'categories': categories,
            'annotations': coco.loadAnns(coco.getAnnIds(imgIds=img_id_list))
        }

        if img_id_list == train_image_ids:
            json_file_path = Path(save_dir, f'{train_type}.json')
        elif img_id_list == val_image_ids:
            json_file_path = Path(save_dir, 'val.json')
        elif img_id_list == test_image_ids:
            json_file_path = Path(save_dir, 'test.json')
        else:
            raise ValueError('img_id_list ERROR!')

        print(f'Saving json to {json_file_path}')
        with open(json_file_path, 'w') as f_json:
            json.dump(img_dict, f_json, ensure_ascii=False, indent=2)

    print('All done!')
split_coco_dataset('./data/annotations.json', './data', [0.95,0.5], True, 2023)


In [ ]:
!rm -rf /kaggle/working/mmpose/work_dir/*

In [ ]:
custom_config = """
_base_ = ['mmpose::_base_/default_runtime.py']

dataset_type = 'CocoDataset'
data_mode = 'topdown'
data_root = './data/'
work_dir = './work_dir'
backend_args = dict(backend='local') # data storage backend

#resume = True
dataset_info = {
    'dataset_name':'Keypoint_reba',
    'classes':'person',
    'paper_info':{
        'author':'Reba pose',
        'title':'Reba Keypoints Detection',
    },
    'keypoint_info':{
    0: {'name': 'Forehead', 'id': 0, 'color': [255, 0, 0], 'type': '', 'swap': ''},
    1: {'name': 'Nose', 'id': 1, 'color': [255, 127, 0], 'type': '', 'swap': ''},
    2: {'name': 'Neck', 'id': 2, 'color': [255, 255, 0], 'type': '', 'swap': ''},
    3: {'name': 'Left Shoulder', 'id': 3, 'color': [0, 255, 0], 'type': '', 'swap': ''},
    4: {'name': 'Right Shoulder', 'id': 4, 'color': [0, 255, 255], 'type': '', 'swap': ''},
    5: {'name': 'Left Elbow', 'id': 5, 'color': [0, 0, 255], 'type': '', 'swap': ''},
    6: {'name': 'Right Elbow', 'id': 6, 'color': [139, 0, 255], 'type': '', 'swap': ''},
    7: {'name': 'Left Wrist', 'id': 7, 'color': [255, 0, 255], 'type': '', 'swap': ''},
    8: {'name': 'Right Wrist', 'id': 8, 'color': [160, 82, 45], 'type': '', 'swap': ''},
    9: {'name': 'Left Hip', 'id': 9, 'color': [255, 0, 0], 'type': '', 'swap': ''},
    10: {'name': 'Center Hip', 'id': 10, 'color': [255, 127, 0], 'type': '', 'swap': ''},
    11: {'name': 'Right Hip', 'id': 11, 'color': [255, 255, 0], 'type': '', 'swap': ''},
    12: {'name': 'Left Knee', 'id': 12, 'color': [0, 255, 0], 'type': '', 'swap': ''},
    13: {'name': 'Right Knee', 'id': 13, 'color': [0, 255, 255], 'type': '', 'swap': ''},
    14: {'name': 'Left Ankle', 'id': 14, 'color': [0, 0, 255], 'type': '', 'swap': ''},
    15: {'name': 'Right Ankle', 'id': 15, 'color': [139, 0, 255], 'type': '', 'swap': ''},
    16: {'name': 'Left Hand', 'id': 16, 'color': [255, 0, 255], 'type': '', 'swap': ''},
    17: {'name': 'Right Hand', 'id': 17, 'color': [160, 82, 45], 'type': '', 'swap': ''}
},
    'skeleton_info': {
    0: {'link': ('Forehead', 'Nose'), 'id': 0, 'color': [255, 0, 0]},
    1: {'link': ('Nose', 'Neck'), 'id': 1, 'color': [255, 0, 0]},
    2: {'link': ('Neck', 'Left Shoulder'), 'id': 2, 'color': [255, 0, 0]},
    3: {'link': ('Neck', 'Right Shoulder'), 'id': 3, 'color': [255, 0, 0]},
    4: {'link': ('Left Shoulder', 'Left Elbow'), 'id': 4, 'color': [255, 0, 0]},
    5: {'link': ('Right Shoulder', 'Right Elbow'), 'id': 5, 'color': [255, 0, 0]},
    6: {'link': ('Left Elbow', 'Left Wrist'), 'id': 6, 'color': [255, 0, 0]},
    7: {'link': ('Right Elbow', 'Right Wrist'), 'id': 7, 'color': [255, 0, 0]},
    8: {'link': ('Neck', 'Left Hip'), 'id': 8, 'color': [255, 0, 0]},
    9: {'link': ('Left Hip', 'Center Hip'), 'id': 9, 'color': [255, 0, 0]},
    10: {'link': ('Center Hip', 'Right Hip'), 'id': 10, 'color': [255, 0, 0]},
    11: {'link': ('Left Hip', 'Left Knee'), 'id': 11, 'color': [255, 0, 0]},
    12: {'link': ('Right Hip', 'Right Knee'), 'id': 12, 'color': [255, 0, 0]},
    13: {'link': ('Left Knee', 'Left Ankle'), 'id': 13, 'color': [255, 0, 0]},
    14: {'link': ('Right Knee', 'Right Ankle'), 'id': 14, 'color': [255, 0, 0]},
    15: {'link': ('Left Wrist', 'Left Hand'), 'id': 15, 'color': [255, 0, 0]},
    16: {'link': ('Right Wrist', 'Right Hand'), 'id': 16, 'color': [255, 0, 0]},
}
}

NUM_KEYPOINTS = len(dataset_info['keypoint_info'])
dataset_info['joint_weights'] = [1.0] * NUM_KEYPOINTS
dataset_info['sigmas'] = [0.025] * NUM_KEYPOINTS

max_epochs = 300
val_interval = 2
train_cfg = {'by_epoch':True,'max_epochs': max_epochs, 'val_begin':2, 'val_interval': val_interval}
train_batch_size = 64
val_batch_size = 32
stage2_num_epochs = 10
base_lr = 0.001
randomness = dict(seed=2023)

param_scheduler = [
    dict( # warmup strategy
        type='LinearLR', begin=0, end=20, start_factor=0.001, by_epoch=False),
    dict( # scheduler
        type='MultiStepLR',
        begin=20,
        end=210,
        milestones=[170, 200],
        gamma=0.1,
        by_epoch=True)
]
optim_wrapper = dict(optimizer=dict(type='Adam', lr=0.001)) # optimizer and initial lr


#auto_scale_lr = dict(base_batch_size=32) # auto scale the lr according to batch size
# automatically scaling LR based on the actual training batch size
# auto_scale_lr = dict(base_batch_size=1024)

# codec settings
# input_size
# input_size
codec = dict(
    type='MSRAHeatmap', input_size=(192, 256), heatmap_size=(48, 64), sigma=2)

model = dict(
    type='TopdownPoseEstimator', # Macro model structure
    data_preprocessor=dict( # data normalization and channel transposition
        type='PoseDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True),
    backbone=dict( # config of backbone
        type='HRNet',
        in_channels=3,
        extra=dict(
            stage1=dict(
                num_modules=1,
                num_branches=1,
                block='BOTTLENECK',
                num_blocks=(4, ),
                num_channels=(64, )),
            stage2=dict(
                num_modules=1,
                num_branches=2,
                block='BASIC',
                num_blocks=(4, 4),
                num_channels=(32, 64)),
            stage3=dict(
                num_modules=4,
                num_branches=3,
                block='BASIC',
                num_blocks=(4, 4, 4),
                num_channels=(32, 64, 128)),
            stage4=dict(
                num_modules=3,
                num_branches=4,
                block='BASIC',
                num_blocks=(4, 4, 4, 4),
                num_channels=(32, 64, 128, 256))),
        init_cfg=dict(
            type='Pretrained', # load pretrained weights to backbone
            checkpoint='https://download.openmmlab.com/mmpose'
            '/pretrain_models/hrnet_w32-36af842e.pth'),
    ),
    head=dict( # config of head
        type='HeatmapHead',
        in_channels=32,
        out_channels=NUM_KEYPOINTS,
        deconv_out_channels=None,
        loss=dict(type='KeypointMSELoss', use_target_weight=True), # config of loss function
        decoder=codec), # get decoder from codec
    test_cfg=dict(
        flip_test=True, # flag of flip test
        flip_mode='heatmap', # heatmap flipping
        shift_heatmap=True,  # shift the flipped heatmap several pixels to get a better performance
    ))

# pipelines
train_pipeline = [
    dict(type='LoadImage', backend_args=backend_args),
    dict(type='GetBBoxCenterScale'),
    dict(type='RandomFlip', direction='horizontal'),
    # dict(type='RandomHalfBody'),
    dict(
        type='RandomBBoxTransform', scale_factor=[0.8, 1.2], rotate_factor=30),
    dict(type='TopdownAffine', input_size=codec['input_size']),
    dict(type='mmdet.YOLOXHSVRandomAug'),
    dict(
        type='Albumentation',
        transforms=[
            dict(type='ChannelShuffle', p=0.5),
            dict(type='CLAHE', p=0.5),
            # dict(type='Downscale', scale_min=0.7, scale_max=0.9, p=0.2),
            dict(type='ColorJitter', p=0.5),
            dict(
                type='CoarseDropout',
                max_holes=4,
                max_height=0.3,
                max_width=0.3,
                min_holes=1,
                min_height=0.2,
                min_width=0.2,
                p=0.5),
        ]),
    dict(type='GenerateTarget', encoder=codec),
    dict(type='PackPoseInputs')
]

val_pipeline = [
    dict(type='LoadImage', backend_args=backend_args),
    dict(type='GetBBoxCenterScale'),
    dict(type='TopdownAffine', input_size=codec['input_size']),
    dict(type='PackPoseInputs')
]

train_pipeline_stage2 = [
    dict(type='LoadImage', backend_args=backend_args),
    dict(type='GetBBoxCenterScale'),
    dict(type='RandomFlip', direction='horizontal'),
    dict(type='RandomHalfBody'),
    dict(
        type='RandomBBoxTransform',
        shift_factor=0.,
        scale_factor=[0.75, 1.25],
        rotate_factor=60),
    dict(type='TopdownAffine', input_size=codec['input_size']),
    dict(type='mmdet.YOLOXHSVRandomAug'),
    dict(
        type='Albumentation',
        transforms=[
            dict(type='Blur', p=0.1),
            dict(type='MedianBlur', p=0.1),
            dict(
                type='CoarseDropout',
                max_holes=1,
                max_height=0.4,
                max_width=0.4,
                min_holes=1,
                min_height=0.2,
                min_width=0.2,
                p=0.5),
        ]),
    dict(type='GenerateTarget', encoder=codec),
    dict(type='PackPoseInputs')
]

# data loaders
train_dataloader = dict(
    batch_size=train_batch_size,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=dataset_info,
        data_mode=data_mode,
        ann_file='trainval.json',
        data_prefix=dict(img='images/'),
        pipeline=train_pipeline,
    ))
val_dataloader = dict(
    batch_size=val_batch_size,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False, round_up=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=dataset_info,
        data_mode=data_mode,
        ann_file='test.json',
        data_prefix=dict(img='images/'),
        pipeline=val_pipeline,
    ))
test_dataloader = val_dataloader

default_hooks = {
    'checkpoint': {'save_best': 'PCK','rule': 'greater','max_keep_ckpts': 2},
    'logger': {'interval': 50}
}

custom_hooks = [
    dict(
        type='EMAHook',
        ema_type='ExpMomentumEMA',
        momentum=0.0002,
        update_buffers=True,
        priority=49),
    dict(
        type='mmdet.PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# evaluators
val_evaluator = [
    dict(type='CocoMetric', ann_file=data_root + 'test.json'),
    dict(type='PCKAccuracy'),
    dict(type='AUC'),
    dict(type='NME', norm_mode='keypoint_distance', keypoint_indices=[0, 1])
]

test_evaluator = val_evaluator
"""
config = './configs/reba_keypoint.py'
with io.open(config, 'w', encoding='utf-8') as f:
    f.write(custom_config)

In [ ]:
!python tools/train.py {config}

In [ ]:
custom_config = """
_base_ = ['mmpose::_base_/default_runtime.py']

dataset_type = 'CocoDataset'
data_mode = 'topdown'
data_root = './data/'
work_dir = './work_dir'
backend_args = dict(backend='local') # data storage backend


dataset_info = {
    'dataset_name':'Keypoint_reba',
    'classes':'person',
    'paper_info':{
        'author':'Reba pos',
        'title':'Reba Keypoints Detection',
    },
    'keypoint_info':{
    0: {'name': 'Forehead', 'id': 0, 'color': [255, 0, 0], 'type': '', 'swap': ''},
    1: {'name': 'Nose', 'id': 1, 'color': [255, 127, 0], 'type': '', 'swap': ''},
    2: {'name': 'Neck', 'id': 2, 'color': [255, 255, 0], 'type': '', 'swap': ''},
    3: {'name': 'Left Shoulder', 'id': 3, 'color': [0, 255, 0], 'type': '', 'swap': ''},
    4: {'name': 'Right Shoulder', 'id': 4, 'color': [0, 255, 255], 'type': '', 'swap': ''},
    5: {'name': 'Left Elbow', 'id': 5, 'color': [0, 0, 255], 'type': '', 'swap': ''},
    6: {'name': 'Right Elbow', 'id': 6, 'color': [139, 0, 255], 'type': '', 'swap': ''},
    7: {'name': 'Left Wrist', 'id': 7, 'color': [255, 0, 255], 'type': '', 'swap': ''},
    8: {'name': 'Right Wrist', 'id': 8, 'color': [160, 82, 45], 'type': '', 'swap': ''},
    9: {'name': 'Left Hip', 'id': 9, 'color': [255, 0, 0], 'type': '', 'swap': ''},
    10: {'name': 'Center Hip', 'id': 10, 'color': [255, 127, 0], 'type': '', 'swap': ''},
    11: {'name': 'Right Hip', 'id': 11, 'color': [255, 255, 0], 'type': '', 'swap': ''},
    12: {'name': 'Left Knee', 'id': 12, 'color': [0, 255, 0], 'type': '', 'swap': ''},
    13: {'name': 'Right Knee', 'id': 13, 'color': [0, 255, 255], 'type': '', 'swap': ''},
    14: {'name': 'Left Ankle', 'id': 14, 'color': [0, 0, 255], 'type': '', 'swap': ''},
    15: {'name': 'Right Ankle', 'id': 15, 'color': [139, 0, 255], 'type': '', 'swap': ''},
    16: {'name': 'Left Hand', 'id': 16, 'color': [255, 0, 255], 'type': '', 'swap': ''},
    17: {'name': 'Right Hand', 'id': 17, 'color': [160, 82, 45], 'type': '', 'swap': ''}
},
    'skeleton_info': {
    0: {'link': ('Forehead', 'Nose'), 'id': 0, 'color': [255, 0, 0]},
    1: {'link': ('Nose', 'Neck'), 'id': 1, 'color': [255, 0, 0]},
    2: {'link': ('Neck', 'Left Shoulder'), 'id': 2, 'color': [255, 0, 0]},
    3: {'link': ('Neck', 'Right Shoulder'), 'id': 3, 'color': [255, 0, 0]},
    4: {'link': ('Left Shoulder', 'Left Elbow'), 'id': 4, 'color': [255, 0, 0]},
    5: {'link': ('Right Shoulder', 'Right Elbow'), 'id': 5, 'color': [255, 0, 0]},
    6: {'link': ('Left Elbow', 'Left Wrist'), 'id': 6, 'color': [255, 0, 0]},
    7: {'link': ('Right Elbow', 'Right Wrist'), 'id': 7, 'color': [255, 0, 0]},
    8: {'link': ('Neck', 'Left Hip'), 'id': 8, 'color': [255, 0, 0]},
    9: {'link': ('Left Hip', 'Center Hip'), 'id': 9, 'color': [255, 0, 0]},
    10: {'link': ('Center Hip', 'Right Hip'), 'id': 10, 'color': [255, 0, 0]},
    11: {'link': ('Left Hip', 'Left Knee'), 'id': 11, 'color': [255, 0, 0]},
    12: {'link': ('Right Hip', 'Right Knee'), 'id': 12, 'color': [255, 0, 0]},
    13: {'link': ('Left Knee', 'Left Ankle'), 'id': 13, 'color': [255, 0, 0]},
    14: {'link': ('Right Knee', 'Right Ankle'), 'id': 14, 'color': [255, 0, 0]},
    15: {'link': ('Left Wrist', 'Left Hand'), 'id': 15, 'color': [255, 0, 0]},
    16: {'link': ('Right Wrist', 'Right Hand'), 'id': 16, 'color': [255, 0, 0]},
}
}

NUM_KEYPOINTS = len(dataset_info['keypoint_info'])
dataset_info['joint_weights'] = [1.0] * NUM_KEYPOINTS
dataset_info['sigmas'] = [0.025] * NUM_KEYPOINTS

max_epochs = 300
val_interval = 2
train_cfg = {'by_epoch'=True,'max_epochs': max_epochs, 'val_begin':2, 'val_interval': val_interval}
train_batch_size = 32
val_batch_size = 32
stage2_num_epochs = 10
base_lr = 0.001
randomness = dict(seed=2023)

optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='Adam', lr=base_lr, weight_decay=0.0001),
    paramwise_cfg=dict(
        norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

param_scheduler = [
    dict(type='LinearLR', start_factor=1.0e-3, by_epoch=False, begin=0, end=600),
    dict(
        type='CosineAnnealingLR',
        eta_min=base_lr * 0.05,
        begin=max_epochs // 2,
        end=max_epochs,
        T_max=max_epochs // 2,
        by_epoch=True,
        convert_to_iter_based=True),
]

# automatically scaling LR based on the actual training batch size
# auto_scale_lr = dict(base_batch_size=1024)

# codec settings
# input_size
# input_size
codec = dict(
    type='MSRAHeatmap', input_size=(192, 256), heatmap_size=(48, 64), sigma=2)

model = dict(
    type='TopdownPoseEstimator', # Macro model structure
    data_preprocessor=dict( # data normalization and channel transposition
        type='PoseDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True),
    backbone=dict( # config of backbone
        type='HRNet',
        in_channels=3,
        extra=dict(
            stage1=dict(
                num_modules=1,
                num_branches=1,
                block='BOTTLENECK',
                num_blocks=(4, ),
                num_channels=(64, )),
            stage2=dict(
                num_modules=1,
                num_branches=2,
                block='BASIC',
                num_blocks=(4, 4),
                num_channels=(32, 64)),
            stage3=dict(
                num_modules=4,
                num_branches=3,
                block='BASIC',
                num_blocks=(4, 4, 4),
                num_channels=(32, 64, 128)),
            stage4=dict(
                num_modules=3,
                num_branches=4,
                block='BASIC',
                num_blocks=(4, 4, 4, 4),
                num_channels=(32, 64, 128, 256))),
        init_cfg=dict(
            type='Pretrained', # load pretrained weights to backbone
            checkpoint='https://download.openmmlab.com/mmpose'
            '/pretrain_models/hrnet_w32-36af842e.pth'),
    ),
    head=dict( # config of head
        type='HeatmapHead',
        in_channels=32,
        out_channels=NUM_KEYPOINTS,
        deconv_out_channels=None,
        loss=dict(type='KeypointMSELoss', use_target_weight=True), # config of loss function
        decoder=codec), # get decoder from codec
    test_cfg=dict(
        flip_test=True, # flag of flip test
        flip_mode='heatmap', # heatmap flipping
        shift_heatmap=True,  # shift the flipped heatmap several pixels to get a better performance
    ))

# pipelines
train_pipeline = [
    dict(type='LoadImage', backend_args=backend_args),
    dict(type='GetBBoxCenterScale'),
    dict(type='RandomFlip', direction='horizontal'),
    # dict(type='RandomHalfBody'),
    dict(
        type='RandomBBoxTransform', scale_factor=[0.8, 1.2], rotate_factor=30),
    dict(type='TopdownAffine', input_size=codec['input_size']),
    dict(type='mmdet.YOLOXHSVRandomAug'),
    dict(
        type='Albumentation',
        transforms=[
            dict(type='ChannelShuffle', p=0.5),
            dict(type='CLAHE', p=0.5),
            # dict(type='Downscale', scale_min=0.7, scale_max=0.9, p=0.2),
            dict(type='ColorJitter', p=0.5),
            dict(
                type='CoarseDropout',
                max_holes=4,
                max_height=0.3,
                max_width=0.3,
                min_holes=1,
                min_height=0.2,
                min_width=0.2,
                p=0.5),
        ]),
    dict(type='GenerateTarget', encoder=codec),
    dict(type='PackPoseInputs')
]

val_pipeline = [
    dict(type='LoadImage', backend_args=backend_args),
    dict(type='GetBBoxCenterScale'),
    dict(type='TopdownAffine', input_size=codec['input_size']),
    dict(type='PackPoseInputs')
]

train_pipeline_stage2 = [
    dict(type='LoadImage', backend_args=backend_args),
    dict(type='GetBBoxCenterScale'),
    dict(type='RandomFlip', direction='horizontal'),
    dict(type='RandomHalfBody'),
    dict(
        type='RandomBBoxTransform',
        shift_factor=0.,
        scale_factor=[0.75, 1.25],
        rotate_factor=60),
    dict(type='TopdownAffine', input_size=codec['input_size']),
    dict(type='mmdet.YOLOXHSVRandomAug'),
    dict(
        type='Albumentation',
        transforms=[
            dict(type='Blur', p=0.1),
            dict(type='MedianBlur', p=0.1),
            dict(
                type='CoarseDropout',
                max_holes=1,
                max_height=0.4,
                max_width=0.4,
                min_holes=1,
                min_height=0.2,
                min_width=0.2,
                p=0.5),
        ]),
    dict(type='GenerateTarget', encoder=codec),
    dict(type='PackPoseInputs')
]

# data loaders
train_dataloader = dict(
    batch_size=train_batch_size,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=dataset_info,
        data_mode=data_mode,
        ann_file='trainval.json',
        data_prefix=dict(img='images/'),
        pipeline=train_pipeline,
    ))
val_dataloader = dict(
    batch_size=val_batch_size,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False, round_up=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=dataset_info,
        data_mode=data_mode,
        ann_file='test.json',
        data_prefix=dict(img='images/'),
        pipeline=val_pipeline,
    ))
test_dataloader = val_dataloader

default_hooks = {
    'checkpoint': {'save_best': 'PCK','rule': 'greater','max_keep_ckpts': 2},
    'logger': {'interval': 50}
}

custom_hooks = [
    dict(
        type='EMAHook',
        ema_type='ExpMomentumEMA',
        momentum=0.0002,
        update_buffers=True,
        priority=49),
    dict(
        type='mmdet.PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# evaluators
val_evaluator = [
    dict(type='CocoMetric', ann_file=data_root + 'test.json'),
    dict(type='PCKAccuracy'),
    dict(type='AUC'),
    dict(type='NME', norm_mode='keypoint_distance', keypoint_indices=[0, 1])
]

test_evaluator = val_evaluator
"""
config = './configs/reba_keypoint.py'
with io.open(config, 'w', encoding='utf-8') as f:
    f.write(custom_config)